In [1]:
# Importing required libraries
import datetime
import time
import pymongo
import pandas as pd
import sys

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient
from pymongo import TEXT

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']

In [3]:
# 
SITE_URL = 'https://www.elespectador.com'

In [4]:
DRIVER_PATH = './geckodriver.exe'

In [5]:
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\ALFRED~1\AppData\Local\Temp/ipykernel_31724/4016513992.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser.
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load.
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis.
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
# Creating a connection to MongoDB to create the health module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-politica']              

In [8]:
# Getting HTML content for news listing page.

news = []
url = '/archivo/politica'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [9]:
for n in news:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [10]:
news

[{'title': '“La Fiscalía no me da garantías”: Petro descarta, por ahora, renunciar al Congreso',
  'relative_path': '/politica/elecciones-colombia-2022/la-fiscalia-no-me-da-garantias-gustavo-petro-descarta-por-ahora-se-renuncia-al-congreso/',
  'datetime': '5 abr 2022  - 9:32 p.\xa0m.',
  'author': 'Redacción Política',
  'summary': 'El candidato presidencial del Pacto Histórico se abstuvo de ahondar en la posible renuncia al Legislativo para centrarse en su campaña presidencial. En respuesta, el fiscal Francisco Barbosa dijo que “es lamentable que pretenda ser presidente de la república cabalgando sobre las instituciones judiciales”.',
  'full_text': 'Gustavo Petro, candidato presidencial del Pacto Histórico y aún senador de la República, se refirió este martes a las peticiones que le han hecho de que renuncie a su curul para dedicarse de lleno a la campaña que viene realizando, y que incluye varios recorridos por el país. A pesar de que han sido varios los sectores que le han hecho l

In [11]:
# Storing extracted information for further analysis.
collection.insert_many(news)

In [12]:
# Creating a connection to MongoDB to create the environment module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-judicial']

In [13]:
# Getting HTML content for news listing page.

newsJudicial = []
url = '/archivo/judicial'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsJudicial.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [14]:
for n in newsJudicial:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [15]:
newsJudicial

[{'title': 'La “ventanilla minera” se la juega toda en el Consejo de Estado',
  'relative_path': '/judicial/la-ventanilla-minera-se-la-juega-toda-en-el-consejo-de-estado/',
  'datetime': '5 abr 2022  - 9:00 p.\xa0m.',
  'author': 'Fernanda Torres Tovar',
  'summary': 'Aunque ya existe una decisión de primera instancia, que tiene suspendidos los términos para entregar licencias mineras, el Consejo de Estado debe analizar si el Gobierno puede seguir otorgándolas o si, por el contrario, se está generando un daño al medio ambiente.',
  'full_text': 'Al menos desde 2013, todo tipo de entidades, privadas y públicas, defensores de derechos humanos, ambientalistas y expertos hicieron una advertencia: la apuesta del gobierno Santos, conocida como la “ventanilla minera”, era una política para explotar la tierra, pero generando una grave afectación para el medio ambiente. El programa también se conoció como la “locomotora minera”, que arrancó motores a todo vapor, hasta que una acción popular se 

In [16]:
# Storing extracted information for further analysis.
collection.insert_many(newsJudicial)

In [17]:
# Creating a connection to MongoDB to create the science module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-economia']

In [18]:
# Getting HTML content for news listing page.

newsEconomia = []
url = '/archivo/economia'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsEconomia.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [19]:
for n in newsEconomia:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [20]:
newsEconomia

[{'title': 'Pensiones en Colombia: qué pasa hoy y hacia dónde debería ir el sistema',
  'relative_path': '/economia/pensiones-en-colombia-que-pasa-hoy-y-hacia-donde-deberia-ir-el-sistema/',
  'datetime': '5 abr 2022  - 9:00 p.\xa0m.',
  'author': 'Karen Vanessa Quintero Martínez',
  'summary': 'Al hablar de pensiones en Colombia parece que por ahora solo hay un acuerdo: es necesario hacer una reforma. En este momento hay varias preguntas sobre la mesa: ¿Cómo aumentar la cobertura?, ¿Cómo crear un sistema más equitativo y más sostenible? Estas son algunas de las propuestas. ',
  'full_text': 'Los problemas del sistema pensional ya están diagnosticados. El principal es la cobertura: hoy solo uno de cada cuatro adultos mayores puede pensionarse en el país, de ahí que esa debe ser la prioridad de la próxima reforma pensional. También hay otros temas como la sostenibilidad del sistema, los subsidios que llegan a quienes menos los necesitan y la competencia entre el Régimen de Prima Media (R

In [21]:
# Storing extracted information for further analysis.
collection.insert_many(newsEconomia)

In [22]:
# Creating a connection to MongoDB to create the technology module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-investigacion']

In [23]:
# Getting HTML content for news listing page.

newsInvestigacion = []
url = '/archivo/investigacion'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsInvestigacion.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [24]:
for n in newsInvestigacion:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [25]:
newsInvestigacion

[{'title': 'Crisis carcelaria en Colombia: ¿Qué hacer con el Inpec?',
  'relative_path': '/investigacion/crisis-carcelaria-en-colombia-que-hacer-con-el-inpec/',
  'datetime': '4 abr 2022  - 1:48 p.\xa0m.',
  'author': 'Nicolás Achury González',
  'summary': 'Los paseos del empresario Carlos Mattos por Bogotá, la fuga de alias “Matamba”, narcotraficante del Clan del Golfo, y el escape de Claudia Peñaranda, protagonista del caso Invima, reafirman un problema carcelario histórico en el país. El Inpec ha tenido escándalos de corrupción, fugas, violencia y complicidad con criminales. Expertos proponen una reforma estructural del sistema penitenciario, pero enfocándose en la política criminal del Estado.',
  'full_text': 'Desde hace 23 años, el inspector Nelson Barrera ha estado en los centros penitenciarios de Bogotá. En la actualidad, trabaja en la cárcel La Modelo y es el presidente del Sindicato de Empleados Unidos Penitenciarios (SEUP). Barrera asegura que “La corrupción sí es tan fuert

In [26]:
# Storing extracted information for further analysis.
collection.insert_many(newsInvestigacion)

In [27]:
# Creating a connection to MongoDB to create the economy module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-bogota']

In [28]:
# Getting HTML content for news listing page.

newsBogota = []
url = '/archivo/bogota'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsBogota.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [29]:
for n in newsBogota:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [30]:
newsBogota

[{'title': 'Motociclistas en Bogotá podrán proponer cambios a decreto que limita su circulación',
  'relative_path': '/bogota/motociclistas-en-bogota-podran-proponer-cambios-a-decreto-que-limita-su-circulacion/',
  'datetime': '5 abr 2022  - 10:11 p.\xa0m.',
  'author': 'Redacción Bogotá',
  'summary': 'El borrador del proyecto del decreto podrá ser modificado de acuerdo con los comentarios que dejen los ciudadanos, entre esos los motociclistas, quienes se han manifestado frente a la restricción de parrillero. Conozca el documento y las medidas de seguridad que contempla el decreto.',
  'full_text': 'En la noche de este 5 de abril, la alcaldesa compartió el borrador del decreto de las medidas de seguridad en Bogotá en el que se menciona la restricción de parrillero que comenzará a regirse partir de este 14 de abril del 2022 y tendrá vigencia hasta el 30 de junio de 2022. Leer: ¿Por qué, a qué hora y dónde protestarán los motociclistas este miércoles en Bogotá? En el documento se lee qu

In [31]:
# Storing extracted information for further analysis.
collection.insert_many(newsBogota)